In [1]:
#In[]
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd
# import statsmodels.api as sm
from tqdm import tqdm
from multiprocessing.dummy import Pool
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# pd.set_option('display.max_columns', 500)

In [2]:
#In[]
crsp_df = pd.read_csv("crsp.zip", compression='zip',header=0,
             parse_dates = ['date','DCLRDT','RCRDDT'],
             dtype={"PRC": np.float64,
                  "VOL": np.float64,
                  "SHROUT": np.float64,
                  "DIVAMT": np.float64,
                  "SPREAD": np.float64,
                  "SHRCD":str,
                  "DISTCD":str,
                  "COMNAM": str,
                  "TICKER": str,
                   'CUSIP': str})
#Data Preprocessing
crsp_df = crsp_df[(crsp_df.SHRCD.isin(('10','11')))]
crsp_df = crsp_df[(crsp_df.date <= '2011-12-31')]

# crsp_df =  crsp_df[( crsp_df['RET'].apply(lambda x: str(x)[-1].isdigit()) )]
# crsp_df['RET'] = crsp_df['RET'].astype('float64')
crsp_df.PRC = crsp_df.PRC.abs()
t = crsp_df.date.dt
crsp_df['year'] = t.year
crsp_df['month'] = t.month
crsp_df.head()


,PERMNO,date,SHRCD,TICKER,COMNAM,CUSIP,DCLRDT,RCRDDT,DISTCD,DIVAMT,PRC,VOL,RET,SHROUT,SPREAD,RETX,year,month
1,10000,1986-01-31,10,OMFGA,OPTIMUM MANUFACTURING INC,68391610,NaT,NaT,NaN,NaN,4.375,1771.000,C,3680.000,0.250,C,1986,1
2,10000,1986-02-28,10,OMFGA,OPTIMUM MANUFACTURING INC,68391610,NaT,NaT,NaN,NaN,3.250,828.000,-0.257143,3680.000,0.250,-0.257143,1986,2
3,10000,1986-03-31,10,OMFGA,OPTIMUM MANUFACTURING INC,68391610,NaT,NaT,NaN,NaN,4.438,1078.000,0.365385,3680.000,0.125,0.365385,1986,3
4,10000,1986-04-30,10,OMFGA,OPTIMUM MANUFACTURING INC,68391610,NaT,NaT,NaN,NaN,4.000,957.000,-0.098592,3793.000,0.250,-0.098592,1986,4
5,10000,1986-05-30,10,OMFGA,OPTIMUM MANUFACTURING INC,68391610,NaT,NaT,NaN,NaN,3.109,1074.000,-0.222656,3793.000,0.094,-0.222656,1986,5


In [3]:
#In[] 
crsp_div_df = crsp_df[['date','CUSIP','DCLRDT','RCRDDT','DISTCD','DIVAMT','PRC','VOL','RET','SHROUT','SPREAD','RETX','year','month']].copy()
# cusips_DISTCD_12 = crsp_div_df[crsp_div_df.DISTCD.apply(lambda x: x[:2]=='12' if isinstance(x, str) else False)].CUSIP.unique()
# crsp_div_df = crsp_div_df[crsp_div_df.CUSIP.isin(cusips_DISTCD_12)]
crsp_div_df.sort_values(by=['CUSIP','date'], ascending=True, inplace=True)
crsp_div_df = crsp_div_df.groupby(by=['CUSIP','date']).agg({
     'DCLRDT': 'last',
     'RCRDDT': 'last',
     'DISTCD': 'last',
     'DIVAMT': 'sum',
     'RET':'last',
     'RETX':'last',
     'VOL': 'last',
     'PRC': 'last',
     'SPREAD': 'last',
     'SHROUT': 'last',
     'year': 'last',
     'month': 'last'})
crsp_div_df['MCAP'] = crsp_div_df['PRC'] * crsp_div_df['SHROUT']
crsp_div_df['TURNOVER'] = crsp_div_df['VOL'] / crsp_div_df['SHROUT']
crsp_div_df['PRC_t-1'] = crsp_div_df.groupby('CUSIP')['PRC'].shift(1)
# crsp_div_df.to_pickle('./crsp_div_df_afterGroupBy.pkl.zip', compression='zip')
crsp_div_df

DCLRDT RCRDDT DISTCD  DIVAMT        RET       RETX  \
CUSIP    date                                                            
00003210 1970-11-30    NaT    NaT   None   0.000          C          C   
         1970-12-31    NaT    NaT   None   0.000   0.111111   0.111111   
         1971-01-29    NaT    NaT   None   0.000  -0.100000  -0.100000   
         1971-02-26    NaT    NaT   None   0.000   0.236111   0.236111   
         1971-03-31    NaT    NaT   None   0.000  -0.123596  -0.123596   
...                    ...    ...    ...     ...        ...        ...   
Y2718360 2008-03-31    NaT    NaT   None   0.000  -0.007712  -0.007712   
         2008-04-30    NaT    NaT   None   0.000   0.002591   0.002591   
         2008-05-30    NaT    NaT   None   0.000   0.014212   0.014212   
         2008-06-30    NaT    NaT   None   0.000  -0.003822  -0.003822   
         2008-07-31    NaT    NaT   None   0.000  -0.001279  -0.001279   

                           VOL    PRC  SPREAD    SHROUT  year  month  \
CUSIP    date                                                          
00003210 1970-11-30    318.000  9.000     NaN  2655.000  1970     11   
         1970-12-31    646.000 10.000     NaN  2655.000  1970     12   
         1971-01-29    636.000  9.000     NaN  2655.000  1971      1   
         1971-02-26   2155.000 11.125     NaN  2655.000  1971      2   
         1971-03-31   2062.000  9.750     NaN  2655.000  1971      3   
...                        ...    ...     ...       ...   ...    ...   
Y2718360 2008-03-31  35972.000  7.720     NaN 49411.000  2008      3   
         2008-04-30  28278.000  7.740     NaN 49411.000  2008      4   
         2008-05-30  34358.000  7.850     NaN 49411.000  2008      5   
         2008-06-30 150341.000  7.820     NaN 49411.000  2008      6   
         2008-07-31  99451.000  7.810     NaN 49411.000  2008      7   

                          MCAP  TURNOVER  PRC_t-1  
CUSIP    date                                      
00003210 1970-11-30  23895.000     0.120      NaN  
         1970-12-31  26550.000     0.243    9.000  
         1971-01-29  23895.000     0.240   10.000  
         1971-02-26  29536.875     0.812    9.000  
         1971-03-31  25886.250     0.777   11.125  
...                        ...       ...      ...  
Y2718360 2008-03-31 381452.920     0.728    7.780  
         2008-04-30 382441.140     0.572    7.720  
         2008-05-30 387876.350     0.695    7.740  
         2008-06-30 386394.020     3.043    7.850  
         2008-07-31 385899.910     2.013    7.820  

[3280235 rows x 15 columns]

In [4]:
#In[] Find months with dividend past year
crsp_div_df.sort_values(by=['CUSIP','date'], ascending=True, inplace=True)
crsp_div_df['freq'] = None
# crsp_div_df['freq'] = crsp_div_df[crsp_div_df.RCRDDT.isna()==False]['DISTCD'].apply(lambda x: x[2:3] if x is not None else None)
crsp_div_df['freq'] = crsp_div_df[crsp_div_df.DIVAMT != 0.0]['DISTCD'].apply(lambda x: x[2] if x is not None and x[:2]=='12' else None)
crsp_div_df['freq'] = crsp_div_df.groupby(by=['CUSIP']).fillna(method='ffill', limit=11)['freq']
crsp_div_df

DCLRDT RCRDDT DISTCD  DIVAMT        RET       RETX  \
CUSIP    date                                                            
00003210 1970-11-30    NaT    NaT   None   0.000          C          C   
         1970-12-31    NaT    NaT   None   0.000   0.111111   0.111111   
         1971-01-29    NaT    NaT   None   0.000  -0.100000  -0.100000   
         1971-02-26    NaT    NaT   None   0.000   0.236111   0.236111   
         1971-03-31    NaT    NaT   None   0.000  -0.123596  -0.123596   
...                    ...    ...    ...     ...        ...        ...   
Y2718360 2008-03-31    NaT    NaT   None   0.000  -0.007712  -0.007712   
         2008-04-30    NaT    NaT   None   0.000   0.002591   0.002591   
         2008-05-30    NaT    NaT   None   0.000   0.014212   0.014212   
         2008-06-30    NaT    NaT   None   0.000  -0.003822  -0.003822   
         2008-07-31    NaT    NaT   None   0.000  -0.001279  -0.001279   

                           VOL    PRC  SPREAD    SHROUT  year  month  \
CUSIP    date                                                          
00003210 1970-11-30    318.000  9.000     NaN  2655.000  1970     11   
         1970-12-31    646.000 10.000     NaN  2655.000  1970     12   
         1971-01-29    636.000  9.000     NaN  2655.000  1971      1   
         1971-02-26   2155.000 11.125     NaN  2655.000  1971      2   
         1971-03-31   2062.000  9.750     NaN  2655.000  1971      3   
...                        ...    ...     ...       ...   ...    ...   
Y2718360 2008-03-31  35972.000  7.720     NaN 49411.000  2008      3   
         2008-04-30  28278.000  7.740     NaN 49411.000  2008      4   
         2008-05-30  34358.000  7.850     NaN 49411.000  2008      5   
         2008-06-30 150341.000  7.820     NaN 49411.000  2008      6   
         2008-07-31  99451.000  7.810     NaN 49411.000  2008      7   

                          MCAP  TURNOVER  PRC_t-1 freq  
CUSIP    date                                           
00003210 1970-11-30  23895.000     0.120      NaN  NaN  
         1970-12-31  26550.000     0.243    9.000  NaN  
         1971-01-29  23895.000     0.240   10.000  NaN  
         1971-02-26  29536.875     0.812    9.000  NaN  
         1971-03-31  25886.250     0.777   11.125  NaN  
...                        ...       ...      ...  ...  
Y2718360 2008-03-31 381452.920     0.728    7.780  NaN  
         2008-04-30 382441.140     0.572    7.720  NaN  
         2008-05-30 387876.350     0.695    7.740  NaN  
         2008-06-30 386394.020     3.043    7.850  NaN  
         2008-07-31 385899.910     2.013    7.820  NaN  

[3280235 rows x 16 columns]

In [5]:
#In[]
crsp_div_df['div_yield'] = crsp_div_df['DIVAMT'].apply(lambda x: 0.0 if pd.isna(x) else x)
crsp_div_df['div_yield'] = list(np.around(crsp_div_df.groupby(by=['CUSIP'])['div_yield'].rolling(window=12, min_periods=0).sum(),decimals=5))
crsp_div_df['div_yield'] = crsp_div_df['div_yield']/crsp_div_df['PRC']
crsp_div_df.loc[crsp_div_df['freq'].isna(),'div_yield'] = None

In [6]:
#In[] load compustat bkvlps and merge to crsp_div_df
compustat_df = pd.read_csv("compustat.zip", compression='zip', header=0,
     parse_dates = ['datadate'],dtype={
          'fyear':np.float64,
          'cusip': str,
          'bkvlps': np.float64
     })
compustat_df = compustat_df[(compustat_df.indfmt != 'FS') & (compustat_df.curcd=='USD')]
compustat_df.dropna(subset=['fyear','cusip'], inplace=True)
compustat_df['CUSIP'] = compustat_df['cusip'].apply(lambda x: None if pd.isna(x) else x[:-1]) 
compustat_df.fyear = compustat_df.fyear.astype(np.int64)
compustat_df['year'] = compustat_df.fyear.apply(lambda x: x+1)
cc_df = pd.merge(crsp_div_df.reset_index(), compustat_df[['CUSIP','year','bkvlps']], on=['CUSIP','year'], how='left')
cc_df.set_index(['CUSIP', 'date'], inplace=True)
cc_df['BM'] =  cc_df['bkvlps']/cc_df['PRC'] 

In [7]:
#In[] Table 1 panel A & B & C
cusips_DISTCD_12 = pd.Series(cc_df[cc_df.DISTCD.apply(lambda x: x[:2]=='12' if isinstance(x, str) else False)].index.get_level_values('CUSIP')).unique()

panel_A = cc_df.loc[pd.IndexSlice[cusips_DISTCD_12,:]]
fil_A = (panel_A["PRC_t-1"] > 5) & (panel_A.freq.isna() == False) & (panel_A.freq != '2')
print("Panel A - Firms with a Dividend in the Past Year")
print(panel_A[fil_A][['MCAP','BM','TURNOVER','SPREAD','div_yield']].describe().T[['count','mean','std','25%','50%','75%']])
print('Number of Firm Months', len(panel_A[fil_A]))
print('Number of Firms', len(pd.Series(panel_A.index.get_level_values('CUSIP')).unique() ))

fil_B = ((cc_df.freq.isna() == True)) & (cc_df["PRC_t-1"] > 5)
panel_B = cc_df[fil_B]
print("Panel B - Firms with No Dividend in the Past Year")
print(panel_B[['MCAP','BM','TURNOVER','SPREAD']].describe().T[['count','mean','std','25%','50%','75%']])
print('Number of Firm Months', len(panel_B))
print('Number of Firms', len(pd.Series(panel_B.index.get_level_values('CUSIP')).unique() ))

#Panel C:

fil_C = (cc_df["PRC_t-1"] >= 5) & (cc_df.freq < '6')
s = cc_df[fil_C].freq
freq_dist = s.groupby(s).count()/ s.groupby(s).count().sum()
any_freq = s.groupby(s).count().sum()/len(cc_df[(cc_df["PRC_t-1"] >= 5)])
panel_C = {
     "Pct of Firm Months with Div pre yr":{
          "Any Freq": any_freq,
          'Monthly': freq_dist[2]*any_freq,
          'Quarterly': freq_dist[3]*any_freq,
          'Semi-Annual': freq_dist[4]*any_freq,
          'Annual': freq_dist[5]*any_freq,
          'Unknown Frequency': (freq_dist[0] + freq_dist[1])*any_freq
     },
     'Pct of Div Obs': {
          'Monthly': freq_dist[2],
          'Quarterly': freq_dist[3],
          'Semi-Annual': freq_dist[4],
          'Annual': freq_dist[5],
          'Unknown Frequency': freq_dist[0] + freq_dist[1]
          }    
}
print("\nPanel C: Distribution of Dividend Frequencies(%)")
print(pd.DataFrame(panel_C)*100)

Panel A - Firms with a Dividend in the Past Year
                count        mean          std       25%        50%        75%
MCAP      1447317.000 1657843.955 10171664.912 30093.250 109540.750 520588.530
BM         933418.000       5.622     1053.530     0.436      0.678      1.020
TURNOVER  1333544.000       0.512        1.039     0.105      0.243      0.565
SPREAD     250836.000       1.173        6.002     0.500      0.750      1.000
div_yield 1447317.000       0.044        0.117     0.018      0.033      0.053
Number of Firm Months 1453655
Number of Firms 9728
Panel B - Firms with No Dividend in the Past Year
              count       mean         std       25%       50%        75%
MCAP     934005.000 750784.969 5649830.420 27749.500 92647.125 351821.080
BM       702398.000      6.806     755.367     0.214     0.420      0.734
TURNOVER 883988.000      1.373       2.216     0.293     0.737      1.684
SPREAD   135874.000      0.866       3.269     0.375     0.500      1.000
Number